In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import unidecode

In [ ]:
data = pd.read_csv('../data/Data_cars.csv')

In [ ]:
price = pd.read_csv('../data/cars_price.csv', encoding='iso-8859-1')

**Remove empty rows**

In [ ]:
price.dropna(subset=['name'], inplace=True)

**Split description col**

In [ ]:
def descriptions_clean(df):
    regex_pattern = r"modele:\s*(?P<modele>.*?(?=,)),\sversion:\s*(?P<version>.*?(?=,)),\spuissance_fiscale:\s*(?P<puissance_fiscale>.*?(?=,)),\sportes:\s*(?P<portes>.*?(?=,)),\soptions:\s*(?P<Descriptions>.*?(?=,)),\scouleur:\s(?P<couleur>.*$)"
    version = []
    spuissance_fiscale = []
    portes = []
    options = []
    couleur = []
    for i in range(df.shape[0]):
        match = re.search(regex_pattern, df.Description[i])
        version.append(match.group(2))
        spuissance_fiscale.append(match.group(3))
        portes.append(match.group(4))
        options.append(match.group(5))
        couleur.append(match.group(6))
    df["version"] = list(map(str.lower, version))
    df["spuissance_fiscale"] = spuissance_fiscale
    df["portes"] = portes
    df["options"] = options
    df["couleur"] = list(map(str.lower, couleur))
    del df["Description"]
    return df

In [ ]:
descriptions_clean(data);

**Rename column**

In [ ]:
price.rename(columns={'energy':'Fuel'}, inplace=True)
price.rename(columns={'transmission':'Gearbox'}, inplace=True)
price.rename(columns={'tax_horsepower':'spuissance_fiscale'}, inplace=True)
price.rename(columns={'date_entry':'Model_year'}, inplace=True)
price.rename(columns={'date_entry':'Model_year'}, inplace=True)

**Fix Fuel col**

In [ ]:
price.Fuel = price.Fuel.apply(lambda x: x.replace('GPL', 'Bicarburation essence GPL')\
                             .replace('GNV', 'Bicarburation essence GNV')\
                             .replace('Électrique', 'Electrique')\
                             .replace('Hybride', 'Hybride diesel électrique') if type(x) == str else None)

**Fix spuissance_fiscale col**

In [ ]:
price.spuissance_fiscale = price.spuissance_fiscale.apply(lambda x: x.replace('CV', '') if type(x) == str else '0')

**Fix Model_year col**

In [ ]:
price.Model_year = price.Model_year.apply(lambda x: int(x.split('/')[-1]))

**Fix Gearbox col**

In [ ]:
price.Gearbox = price.Gearbox.apply(lambda x: x.replace('Automatique', 'automatique').replace('Manuelle', 'mécanique'))

**Fix portes col** *i apply on data*

In [ ]:
data.portes = data.portes.apply(lambda x: x.replace('.0', ''))

**Fix price bonus/malus**

In [ ]:
price.starting_price = price.starting_price.apply(lambda x: re.search('\d+', x).group(0))

In [ ]:
price['bonus/malus'].fillna('0', inplace=True)
price['bonus/malus'].replace(['n.a.', '\x80', ''], '0', inplace=True)
price['bonus/malus'] = price['bonus/malus'].apply(lambda x: re.search('-?\d+', x).group(0))

**Make and Model unaccented**

In [ ]:
price['name'] = price['name'].apply(lambda x: unidecode.unidecode(x))

**Convert to lower case make the join easier**

In [ ]:
price.name = price.name.str.lower()
data.Make = data.Make.str.lower()
data.Model = data.Model.str.lower()

**Split col name in two columns Make and Model**

In [ ]:
brands = list(data.Make.unique())

In [ ]:
def get_brand_name(x):
    if ' '.join(x.split()[0:2]) in brands:
        return ' '.join(x.split()[0:2])
    return x.split()[0]

In [ ]:
def get_model_name(x):
    if ' '.join(x.split()[0:2]) in brands:
        return ' '.join(x.split()[2:])
    return ' '.join(x.split()[1:])

In [ ]:
price['Make'] = price.name.apply(lambda x: get_brand_name(x) )
price['Model'] = price.name.apply(lambda x: get_model_name(x) )

**Create portes col**

In [ ]:
price['portes'] = price.version.apply(lambda x: x.split('(')[-1].replace(')', '').replace('p.', ''))

**Model errors**

In [ ]:
def price_clean_audi(x):
    if x.split()[0] == 'tts':
        return 'tt'
    return x.split()[0]

In [ ]:
def data_clean_audi(x):
    if x == 'a6 allroad':
        return 'a6'
    elif x == 'a4 allroad':
        return 'a4'
    elif x == 'tt rs':
        return 'tt'
    elif x == 'allroad':
        return 'a6'
    else:
        return x

In [ ]:
def price_clean_ford(x):
    if x.split()[0] == 'grand':
        return ' '.join(x.split()[0:2])
    return x.split()[0]

In [ ]:
def data_clean_ford(x):
    if x == 'ka+':
        return 'ka'
    elif x.split()[0] == 'grand':
        return ' '.join(x.split()[0:2])
    return x.split()[0]

In [ ]:
def price_clean_citroen(x):
    if x.split()[0] == 'grand':
        return ' '.join(x.split()[0:2])
    return x.split()[0]

In [ ]:
def data_clean_citroen(x):
    x = x.replace('picasso', 'spacetourer')
    if x == 'ds 4':
        return 'ds4'
    elif x.split()[0] == 'grand':
        return ' '.join(x.split()[0:2])
    return x.split()[0]

**Primary Funcs**

In [ ]:
def clean_model_price(x, clean_type):
    if clean_type == 'audi':
        return price_clean_audi(x)
    elif clean_type == 'ford':
        return price_clean_ford(x)
    elif clean_type == 'citroen':
        return price_clean_citroen(x)

In [ ]:
def clean_model_data(x, clean_type):
    if clean_type == 'audi':
        return data_clean_audi(x)
    elif clean_type == 'ford':
        return data_clean_ford(x)
    elif clean_type == 'citroen':
        return data_clean_citroen(x)

In [ ]:
def replace_col(model, fun_clean, df):
    tmp = df[df.Make == model].Model.apply(lambda x: fun_clean(x, model))
    keys = list(tmp.keys())
    vals = list(tmp.values)
    df.loc[keys, 'Model'] = vals

**Replace in Models**

In [ ]:
replace_col('audi', clean_model_price, price)
replace_col('audi', clean_model_data, data)

In [ ]:
replace_col('ford', clean_model_price, price)
replace_col('ford', clean_model_data, data)

In [ ]:
replace_col('citroen', clean_model_price, price)
replace_col('citroen', clean_model_data, data)

**Fix small error**

In [ ]:
data.loc[17541, 'Model_year'] = 2019 # year 5019

**Merge**

In [ ]:
#### merged = data.merge(price, how='outer', on=['Make', 'Model', 'Fuel', 'Gearbox', 'spuissance_fiscale', 'Model_year', 'portes'], indicator=True)

In [ ]:
outliers = list(data[(data['Model_year']<2000)].index)
len(outliers), len(data)

In [ ]:
data.drop(index=outliers, inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
def find_price_car(item, price):
    
if_puiss_fisc = price["spuissance_fiscale"] == item['spuissance_fiscale']
if_model_year = price['Model_year'] == item['Model_year']
if_portes = price['portes'] == item['portes']
if_gearbox = price['Gearbox'] == item['Gearbox']
if_fuel = price['Fuel'] == item['Fuel']

car_0 = price[(price['Make'] == item['Make']) & (price['Model'] == item['Model'])]

if len(car_0.where(if_fuel | if_gearbox | if_puiss_fisc | if_model_year | if_portes, inplace=False).dropna()) > 0:
    car_0 = car_0.where(if_puiss_fisc | if_model_year | if_portes, inplace=False).dropna()
        
    car_0.reset_index(drop=True, inplace=True)
    
    if len(car_0) == 0:
        return None
    
    for index, row in car_0.iterrows():
        car_0.loc[index, 'score'] = fuzz.token_set_ratio(row['version'], item['version'])
    
    return car_0.loc[car_0.score.argmax()]

In [ ]:
for i in range(len(data)):
    print(i, end='\r')
    item = data.iloc[i]
    price_car = find_price_car(item, price)
    if price_car is not None:
        data.loc[i, 'starting_price'] = price_car['starting_price']
        data.loc[i, 'bonus_malus'] = price_car['bonus/malus']
        data.loc[i, 'puiss_scrap'] = price_car['spuissance_fiscale']
        data.loc[i, 'portes_scrap'] = price_car['portes']

**On elimine les voitures trop cher**

In [ ]:
data[data.Price < 4*pow(10,4)].Price.plot.hist()

In [ ]:
data = df[(data.Price < 3*pow(10,4)) & (data.Price > 5000)]

**On elimine les voitures qui sont plus cher que le prix initial**

In [ ]:
df['Price/Starting Price'] = list(map(lambda x, y: 1 if x > y else 0, df.Price, df.starting_price ))

In [ ]:
df = df[df['Price/Starting Price'] == 0]

**On elimine les voitures les voitures qui sont trop puissantes**

In [ ]:
df = df[df.spuissance_fiscale < 30]

**Conversion de colonnes string en numérique**

In [ ]:
df.puiss_scrap = df.puiss_scrap.fillna(0)

In [ ]:
df.puiss_scrap = df.puiss_scrap.astype(int)

In [ ]:
df.portes_scrap = df.portes_scrap.astype(float)

In [ ]:
df.to_csv('init_price_cleaned.csv', index=False)